In [ ]:
# This is necessary to recognize the modules
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

import asyncio
from decimal import Decimal
from core.data_sources import CLOBDataSource
from core.features.candles.volatility import VolatilityConfig
from research_notebooks.xtreet_bb.utils import generate_config, dump_dict_to_yaml

clob = CLOBDataSource()

In [ ]:
# Screener parameters
CONNECTOR_NAME = "binance_perpetual"
INTERVALS = ["1m"]
DAYS = 7
FETCH_CANDLES = False
BATCH_CANDLES_REQUEST = 15
SLEEP_REQUEST = 60.0
VOLUME_THRESHOLD = 0.3  # From percentile VOLUME_THRESHOLD to 1
VOLATILITY_THRESHOLD = 0.4  # From percentile VOLATILITY_THRESHOLD to 1

# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

VOLATILITY_WINDOW = 60  # In bars
VOLUME_FAST_WINDOW = 20  # No se usa
VOLUME_SLOW_WINDOW = 100  # No se usa

# Config generation
TOTAL_AMOUNT = 300  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_EXECUTORS_PER_SIDE = 1  # Maximum number of executors per side
COOLDOWN_TIME = 0
LEVERAGE = 20 # Should be for each trading pair
TIME_LIMIT = 60 * 60 * 24
BOLLINGER_LENGTHS = [50, 100, 200]
BOLLINGER_STDS = [1.4, 1.8, 2.2]
SL_STD_MULTIPLIER = 2
TS_DELTA_MULTIPLIER = 0.2

# Config filtering
MIN_DISTANCE_BETWEEN_ORDERS = 0.01
MAX_TS_SL_RATIO = 0.5

# Download data
- Get trading rules
- Get candles for the last x days

In [ ]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
    .get_all_trading_pairs()

if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAME,
        trading_pair=trading_pair,
        interval=interval,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch for interval in INTERVALS]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))
candles = [value for key, value in clob.candles_cache.items() if key[2] in INTERVALS and key[0] == CONNECTOR_NAME]

In [ ]:
from core.features.candles.volume import VolumeConfig
from research_notebooks.xtreet_bb.utils import generate_screener_report

screner_report = generate_screener_report(
    candles=candles,
    trading_rules=trading_rules,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_FAST_WINDOW, long_window=VOLUME_FAST_WINDOW))
screner_report["url"] = screner_report["trading_pair"].apply(lambda x: f"https://www.binance.com/en/trade/{x}")
screner_report.sort_values("mean_natr", ascending=False, inplace=True)
screner_report

In [ ]:
# Calculate the 20th percentile (0.2 quantile) for both columns
natr_percentile = screner_report['mean_natr'].quantile(VOLATILITY_THRESHOLD)
volume_percentile = screner_report['average_volume_per_hour'].quantile(VOLUME_THRESHOLD)

# Filter the DataFrame to get observations where mean_natr is greater than its 20th percentile
# and average_volume_per_hour is greater than its 20th percentile
screener_top_markets = screner_report[
    (screner_report['mean_natr'] > natr_percentile) &
    (screner_report['average_volume_per_hour'] > volume_percentile) &
    (screner_report["price_step_pct"] < MAX_PRICE_STEP)
]

# Display the filtered DataFrame
screener_top_markets[["trading_pair", "mean_natr", "average_volume_per_hour", "price_step_pct", "url"]]

In [ ]:
strategy_configs = generate_config(
    connector_name=CONNECTOR_NAME,
    intervals=INTERVALS,
    screener_top_markets=screener_top_markets,
    candles=candles,
    total_amount=TOTAL_AMOUNT,
    max_executors_per_side=MAX_EXECUTORS_PER_SIDE,
    cooldown_time=COOLDOWN_TIME,
    leverage=LEVERAGE,
    time_limit=TIME_LIMIT,
    bb_lengths=BOLLINGER_LENGTHS,
    bb_stds=BOLLINGER_STDS,
    sl_std_multiplier=SL_STD_MULTIPLIER,
    min_distance_between_orders=MIN_DISTANCE_BETWEEN_ORDERS,
    max_ts_sl_ratio=MAX_TS_SL_RATIO,
    ts_delta_multiplier=TS_DELTA_MULTIPLIER,
)

In [ ]:
len(set([config["trading_pair"] for config in strategy_configs]))

In [ ]:
len(strategy_configs)

In [ ]:
# DCA amounts
for config in strategy_configs:
    dump_dict_to_yaml("configs/", config)